In [1]:
import pandas as pd
from itertools import product
from tqdm.notebook import tqdm

import sys, os

sys.path.append(os.path.abspath('..'))
%load_ext autoreload
%autoreload 2
from modules.storage import (
    get_demand_model_data,
    get_demand_orig_dest_model_data,
)
from modules.config import *
from modules.svm import *

In [2]:
all_possible_metas = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10, 100],  'gamma': [-1],                          'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['rbf'],    'C': [0.1, 1, 10, 100],  'gamma': [0.1, 0.01, 0.001, 0.0001],    'degree': [-1],         'max_iter': [1000000]},
    {'kernel': ['poly'],   'C': [1, 10, 100, 1000], 'gamma': [-1],                          'degree': [2, 3, 4, 5], 'max_iter': [1000000]}
]

In [3]:
execute_stage(
    'first_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    TUNE_H3_RESOLUTION,
    TUNE_TIME_INTERVAL_LENGTH,
    all_possible_metas,
    get_demand_model_data,
)

  0%|          | 0/3 [00:00<?, ?it/s]

c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


h3: 8 | t:6 | - linear ✓
h3: 8 | t:6 | - rbf ✓
h3: 8 | t:6 | - poly ✓


In [4]:
results = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False).head(2)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,param_max_iter,params,...,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score,n_iter,h3_res,time_interval_length,param_gamma,param_degree
52,2,29997,610.163423,4.774594,52.755195,4.022380,100.0,poly,1000000,"{'C': 100.0, 'degree': 5.0, 'gamma': None, 'ke...",...,-5.297303,-5.170202,-5.103442,-5.184127,0.064518,0,8,6,NaN,5.0
53,2,29997,726.315229,11.302701,93.564156,4.522072,100.0,poly,1000000,"{'C': 100.0, 'degree': 3.0, 'gamma': None, 'ke...",...,-9.430366,-9.380489,-9.514133,-9.401522,0.077954,0,8,6,NaN,3.0


In [5]:
results.sort_values(by='mean_test_score', ascending=False).groupby('param_kernel')[[
    'mean_test_score',
    'param_C',
    'param_degree',
    'param_gamma',
]].first()

,mean_test_score,param_C,param_degree,param_gamma
param_kernel,,,,
linear,-53.220806,100.0,NaN,NaN
poly,-17.281016,100.0,5.0,NaN
rbf,-18.259941,100.0,NaN,0.01


In [6]:
results.groupby('param_kernel').mean_fit_time.sum() / 60

param_kernel
linear    41.028646
poly      41.674097
rbf       36.527084
Name: mean_fit_time, dtype: float64

In [7]:
resolutions = list(product(PREDICTIVE_H3_RESOLUTIONS, CALC_TIME_INTERVAL_LENGTHS))
resolutions = resolutions + (ADDITIONAL_PREDICTIVE_RESOLUTIONS)
resolutions

[(7, 1), (7, 2), (7, 6), (7, 24), (8, 1), (8, 2), (8, 6), (8, 24), (9, 24)]

In [8]:
for h3_res, time_interval_length in tqdm(resolutions):
    execute_stage(
        'second_stage',
        SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
        SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
        h3_res,
        time_interval_length,
        all_possible_metas,
        get_demand_model_data,
    )

  0%|          | 0/9 [00:00<?, ?it/s]

c:\Dev\miniconda\envs\AAA_MAGMA\lib\site-packages\sklearn\svm\_base.py:301: ConvergenceWarning: Solver terminated early (max_iter=1000000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


In [9]:
results = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results.sort_values(by=['mean_test_score'], ascending=False)

,iter,n_resources,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_degree,param_kernel,param_max_iter,...,std_train_score,n_iter,h3_res,time_interval_length,test_mse,test_rmse,test_mae,test_non_zero_mape,test_zero_accuracy,param_gamma
4,0,30000,161.777926,8.356687,35.571328,4.316158,1000.0,5.0,poly,1000000,...,0.000033,65262,8,1,0.509334,0.713677,0.278801,0.614778,0.913479,NaN
0,0,30000,71.569414,4.031449,12.446590,4.398620,1000.0,5.0,poly,1000000,...,0.055264,86750,7,1,3.128297,1.768699,0.561854,0.474522,0.887161,NaN
5,0,30000,217.941595,12.112711,41.003597,8.799975,1000.0,5.0,poly,1000000,...,0.003895,185647,8,2,4.096558,2.023995,0.713419,0.598321,0.844196,NaN
8,0,30000,734.771367,37.549914,127.169526,31.679114,1000.0,5.0,poly,1000000,...,0.101142,659582,9,24,13.215704,3.635341,1.673645,0.512333,0.808409,NaN
6,0,30000,290.607039,5.993957,43.448844,7.690702,1000.0,5.0,poly,1000000,...,0.364513,550874,8,6,21.517107,4.638654,1.746573,0.471780,0.787242,NaN
1,0,30000,127.594942,5.196947,14.824186,4.430147,1000.0,5.0,poly,1000000,...,0.057779,269218,7,2,26.892334,5.185782,1.692824,0.422806,0.795999,NaN
7,0,30000,365.084320,14.383867,40.635177,12.322926,1000.0,5.0,poly,1000000,...,0.612568,849278,8,24,52.350069,7.235335,2.825611,0.297238,0.800737,NaN
2,0,30000,298.942540,5.409172,19.224739,4.357530,1000.0,5.0,poly,1000000,...,0.847525,1000000,7,6,115.649689,10.754055,3.782605,0.256605,0.689350,NaN
3,0,8797,16.271286,0.278146,1.749223,0.353983,1000.0,5.0,poly,1000000,...,4.268758,157534,7,24,571.556351,23.907245,8.444464,0.213361,0.733235,NaN


In [10]:
results_first_stage = pd.read_parquet(SVM_FIRST_STAGE_DEMAND_RESULTS_PATH)
results_second_stage = pd.read_parquet(SVM_SECOND_STAGE_DEMAND_RESULTS_PATH)
results_first_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_first_stage_demand_results.parquet"))
results_second_stage.to_parquet(os.path.join(REPO_DATA_DIR_PATH, f"{str(SVM_MAX_TRAIN_SET_SIZE)}_svm_second_stage_demand_results.parquet"))

In [11]:
execute_stage(
    'second_stage',
    SVM_FIRST_STAGE_DEMAND_RESULTS_PATH,
    SVM_SECOND_STAGE_DEMAND_RESULTS_PATH,
    int(f"{h3_res}{h3_res}"),
    time_interval_length,
    all_possible_metas,
    lambda _, __, : get_demand_orig_dest_model_data(),
)